In [ ]:
import os
os.chdir(os.path.pardir)

import numpy as np
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
from gglasso.helper.data_generation import group_power_network, sample_covariance_matrix
from gglasso.problem import glasso_problem
from gglasso.helper.basic_linalg import adjacency_matrix
from gglasso.helper.utils import sparsity, zero_replacement, normalize, log_transform

# Import preprocessed soil data

In [ ]:
soil = pd.read_table('~/soil_116.csv', sep=',')
soil.head()

In [ ]:
X_soil = np.array(soil)
X_soil.shape

# Log-transform

In [ ]:
X = normalize(X_soil)

Note: X is scaled with geometric mean, and should be a dataframe of form (p,N)

In [ ]:
X = log_transform(pd.DataFrame(X.T))

# Calculate covariance and scale to correlations

In [ ]:
S0 = np.cov(X.values, bias = True)

In [ ]:
# scale covariances to correlations
scale = np.tile(np.sqrt(np.diag(S0)),(S0.shape[0],1))
scale = scale.T * scale

S = S0 / scale

# GGLasso problem

In [ ]:
P = glasso_problem(S, X.shape[1], reg_params = {'lambda1': 0.05}, latent = False, do_scaling = False)
print(P)

In [ ]:
lambda1_range = np.logspace(0, -3, 20)
modelselect_params = {'lambda1_range': lambda1_range}

P.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = 1)

# regularization parameters are set to the best ones found during model selection
print(P.reg_params)

In [ ]:
sol = P.solution.precision_
P.solution.calc_adjacency()


plt.figure()
G1 = nx.from_numpy_array(P.solution.adjacency_)
nx.draw_spring(G1, node_color = "darkblue", edge_color = "darkblue", font_color = 'white', with_labels = True)